In [6]:
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from gensim import corpora
from gensim.models import LdaModel, TfidfModel
from gensim.models.phrases import Phrases, Phraser
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Read the Excel file
file_path = "/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data/ICML.xlsx"
df = pd.read_excel(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data/ICML.xlsx'

In [3]:
# Extract keywords column
keywords_column = 'keywords'

# Extract the keywords column, remove empty values (NaN), and convert to a list
keywords_list = df[keywords_column].dropna().tolist()

In [4]:
# Split all keywords into individual words
all_keywords = []
for keywords in keywords_list:
    # Keywords are separated by commas
    split_keywords = [kw.strip() for kw in keywords.split(",")]
    all_keywords.extend(split_keywords)  # Add the split keywords to the total list

# Count word frequency
keyword_freq = Counter(all_keywords)  # Use Counter to count the occurrences of each keyword

In [5]:
# Print frequency statistics
print("Keyword Frequency Statistics:")
for keyword, freq in keyword_freq.most_common():  # Sort by frequency from high to low
    print(f"{keyword}: {freq}")  # Print the keyword and its frequency

Keyword Frequency Statistics:
Algorithms: 26
Reinforcement Learning and Planning: 19
Reinforcement Learning: 15
Supervised Learning: 12
Bandits: 9
DL: Robustness: 8
Active Learning: 7
Online Learning / Bandits: 7
Fairness: 7
Other: 7
Deep Learning - General: 7
Multitask and Meta-learning: 7
Deep Learning: 7
T: Optimization: 7
APP: Computer Vision: 7
DL: Theory: 7
Equity: 6
Large Scale Learning and Big Data: 6
Architectures: 6
Transparency and Interpretability: 6
Adversarial Examples: 6
MISC: Unsupervised and Semi-supervised Learning: 6
RL: Deep RL: 6
DL: Algorithms: 6
MISC: Transfer: 6
Online Learning: 5
Optimization: 5
Deep Reinforcement Learning: 5
Deep Generative Models: 5
Convex Optimization: 5
Learning Theory: 5
Speech and Dialog: 5
OPT: Discrete and Combinatorial Optimization: 5
SA: Trustworthy Machine Learning: 5
T: Deep Learning: 5
MISC: Scalable Algorithms: 5
T: Everything Else: 5
DL: Other Representation Learning: 5
DL: Self-Supervised Learning: 5
DL: Everything Else: 5
and B

In [6]:
# Save the frequency statistics to a new Excel file
output_file_path = "/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/keyword_frequency.xlsx"
# Convert the frequency statistics to a DataFrame and save as an Excel file
pd.DataFrame(keyword_freq.most_common(), columns=["Keyword", "Frequency"]).to_excel(output_file_path, index=False)

print(f"\nThe frequency statistics have been saved to: {output_file_path}")


The frequency statistics have been saved to: /Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/keyword_frequency.xlsx


This process involves using Latent Dirichlet Allocation (LDA) to extract the main keywords from the article titles, keywords, and abstracts. Then, the articles are classified into categories based on the keyword matching to the ML research categories defined by ICML.

In [7]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [8]:
# Define category keywords
CATEGORIES_KEYWORDS = {
    "General Machine Learning": [
        "General Machine Learning", "active learning", "query strategy", "uncertainty sampling", "clustering", "k-means",
        "hierarchical clustering", "spectral clustering", "online learning", "regret minimization",
        "ranking", "learning to rank", "supervised learning", "classification", "regression",
        "semi-supervised learning", "label propagation", "graph-based methods", 
        "self-supervised learning", "contrastive learning", "masked modeling",
        "time series analysis", "forecasting", "temporal models"
    ],
    "Deep Learning": [
        "Deep Learning", "architecture", "cnn", "rnn", "transformer", "gnn", "generative model", "gan",
        "vae", "diffusion model", "autoregressive model", "normalizing flow",
        "theory", "generalization", "double descent", "neural tangent kernel", 
        "attention mechanism", "pretraining", "transfer learning"
    ],
    "Evaluation": [
        "Evaluation", "methodology", "cross-validation", "evaluation metric", "meta studies", "benchmarking",
        "replicability", "reproducibility", "validity", "dataset shift",
        "human-in-the-loop", "user feedback", "interactive learning"
    ],
    "Theory of Machine Learning": [
        "Theory of Machine Learning", "statistical learning theory", "vc dimension", "pac learning", "bandits", 
        "multi-armed bandit", "contextual bandit", "game theory", "multi-agent learning", 
        "decision theory", "bayesian decision", "cost-sensitive learning", "information theory"
    ],
    "Machine Learning Systems": [
        "Machine Learning Systems", "improved implementation", "system design", "scalability", "distributed training",
        "hardware", "gpu", "tpu", "libraries", "tensorflow", "pytorch", "jax",
        "distributed methods", "model parallelism", "data parallelism"
    ],
    "Optimization": [
        "Optimization", "convex optimization", "gradient descent", "dual methods", "non-convex optimization",
        "landscape analysis", "matrix methods", "low-rank approximation",
        "tensor methods", "tensor decomposition", "stochastic", "sgd", "adam",
        "online", "adaptive methods", "non-smooth", "subgradient", 
        "composite", "proximal algorithm"
    ],
    "Probabilistic Methods": [
        "Probabilistic Methods", "bayesian methods", "posterior inference", "bayesian neural network",
        "graphical models", "bayesian network", "markov random field",
        "monte carlo methods", "mcmc", "gibbs sampling", "hamiltonian monte carlo",
        "variational inference", "uncertainty quantification"
    ],
    "Reinforcement Learning": [
        "Reinforcement Learning", "decision and control", "policy optimization", "planning", "model-based rl",
        "hierarchical rl", "temporal abstraction", "robotics", "imitation learning",
        "multi-agent rl", "exploration", "intrinsic motivation"
    ],
    "Trustworthy Machine Learning": [
        "Trustworthy Machine Learning", "causality", "structural causal model", "counterfactual", "fairness", 
        "equalized odds", "demographic parity", "interpretability", "shap", "lime",
        "privacy", "differential privacy", "federated learning", "robustness",
        "adversarial example", "certified defense", "safety", "safe rl", "value alignment"
    ],
    "Application-Driven Machine Learning": [
        "Application-Driven Machine Learning", "innovative techniques", "application", "problems", "real-world datasets", "healthcare", 
        "electronic health records", "medical imaging", "physical sciences", "climate modeling",
        "biosciences", "genomics", "drug discovery", "social sciences", 
        "computational social science", "network analysis", "sustainability", "climate", 
        "carbon modeling", "energy forecasting"
    ]
}

In [9]:
# Data preprocessing function
def preprocess_text(text):  # Includes conversion to lowercase, punctuation removal, tokenization, stop word removal, and lemmatization.
    
    if pd.isna(text):  # Check if the text is empty
        return []
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    words = word_tokenize(text)
    # Remove stopwords and perform lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and word.isalpha()]
    return words

In [10]:
# Read the Excel file
file_path = '/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data/ICML.xlsx'
df = pd.read_excel(file_path)  # Read the Excel file

# Preprocess title, keywords, abstract
df['processed_title'] = df['title'].apply(preprocess_text)  
df['processed_keywords'] = df['keywords'].apply(preprocess_text)  
df['processed_abstract'] = df['abstract'].apply(preprocess_text)  

# Combine processed text
df['processed_text'] = df['processed_title'] + df['processed_keywords'] + df['processed_abstract']

In [11]:
# Extract bigrams
bigram = Phrases(df['processed_text'], min_count=5, threshold=10)  # Initialize the bigram model
bigram_phraser = Phraser(bigram)  # Convert bigram model to Phraser object
df['processed_text'] = [bigram_phraser[doc] for doc in df['processed_text']]  # Apply the bigram model to each article

In [12]:
# Create dictionary and corpus
dictionary = corpora.Dictionary(df['processed_text'])  # Create dictionary
corpus = [dictionary.doc2bow(text) for text in df['processed_text']]  # Convert text to bag-of-words model

In [13]:
# Create TF-IDF model
tfidf = TfidfModel(corpus)  # Initialize TF-IDF model
corpus_tfidf = tfidf[corpus]  # Convert corpus to TF-IDF representation

In [14]:
# Build LDA model
lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)  # Build LDA model

In [15]:
# Extract core keywords from each article
def get_keywords(doc, dictionary, lda_model, topn=5):  # Extract core keywords from each article.
    bow = dictionary.doc2bow(doc)  # Convert document to bag-of-words model
    topic_distribution = lda_model.get_document_topics(bow)  # Get document's topic distribution
    topic_id = max(topic_distribution, key=lambda x: x[1])[0]  # Find the most relevant topic
    topic_words = lda_model.show_topic(topic_id, topn=topn)  # Get core keywords for that topic
    return [word for word, _ in topic_words]  # Return list of keywords

df['core_keywords'] = df['processed_text'].apply(lambda x: get_keywords(x, dictionary, lda_model))  # Extract core keywords for each article

In [16]:
# Define category assignment function
def assign_category(core_keywords, categories_keywords):  # Assign categories based on core keywords.
    
    category_scores = {category: 0 for category in categories_keywords}  # Initialize category scores
    for keyword in core_keywords:
        for category, keywords in categories_keywords.items():
            if keyword in keywords:  # If keyword matches category keywords
                category_scores[category] += 1  # Increase the score for that category
    if sum(category_scores.values()) == 0:  # If no matching categories
        return "Uncategorized"
    return max(category_scores, key=category_scores.get)  # Return the category with the highest score

# Assign categories to each article
df['category'] = df['core_keywords'].apply(lambda x: assign_category(x, CATEGORIES_KEYWORDS)) 

In [17]:
# Save the results to a new Excel file
output_file_path = '/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/ICML_LDA_categories.xlsx'
df.to_excel(output_file_path, index=False)  # Save the results to an Excel file

print(f"The results have been saved to: {output_file_path}")

The results have been saved to: /Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/ICML_LDA_categories.xlsx


However, due to the relatively common words extracted by LDA (such as "method"), which cannot be recognized, the proportion of uncategorized articles exceeds 60%. Therefore, the approach involves performing keyword searches on the article titles and keywords. If an article cannot be categorized, the abstract will be analyzed to avoid excessive labeling.

In [18]:
# Adding New Classification Method (ML Survey Paper Classification Method)
ADDITIONAL_CATEGORIES_KEYWORDS = {
    "Supervised Learning": [
        "Supervised Learning", "labeled data", "classification", "regression", "training set", "test set", 
        "loss function", "cross-validation", "overfitting", "support vector machine", 
        "decision tree", "random forest", "logistic regression", "gradient boosting"
    ],
    "Unsupervised Learning": [
        "Unsupervised Learning", "unlabeled data", "clustering", "dimensionality reduction", 
        "pattern recognition", "density estimation", "anomaly detection", "k-means", 
        "hierarchical clustering", "spectral clustering", "PCA", "t-SNE", "autoencoders"
    ],
    "Semi-Supervised Learning": [
        "Semi-Supervised Learning", "partially labeled data", "self-training", "pseudo-labeling", 
        "transductive learning", "graph-based learning", "semi-supervised SVM", "Mean Teacher", 
        "MixMatch", "FixMatch", "label propagation", "consistency regularization"
    ],
    "Multi-task Learning": [
        "Multi-task Learning", "shared representation", "task similarity", "auxiliary tasks", 
        "hard parameter sharing", "soft parameter sharing", "transfer learning", 
        "meta-learning", "cross-stitch networks"
    ],
    "Ensemble Learning": [
        "Ensemble Learning", "model combination", "boosting", "bagging", "stacking", 
        "random forest", "XGBoost", "gradient boosting", "AdaBoost", "voting methods", 
        "weak learners", "weighted averaging"
    ],
    "Instance Based Learning": [
        "Instance Based Learning", "lazy learning", "nearest neighbors", "similarity measure", 
        "distance metric", "case-based reasoning", "locally weighted learning", 
        "k-nearest neighbors", "memory-based learning"
    ]
}

In [19]:
# Classification Function
def classify_text(title, keywords, abstract, category_dict):
    # First, match keywords from title and keywords
    combined_title_keywords = f"{title} {keywords}".lower()  # Combine title and keywords in lowercase
    matched_categories = [cat for cat, kw_list in category_dict.items() if any(kw.lower() in combined_title_keywords for kw in kw_list)]  # Match categories

    # If no categories matched, proceed to match keywords from the abstract
    if not matched_categories:
        combined_abstract = abstract.lower()  # Convert abstract to lowercase
        matched_categories = [cat for cat, kw_list in category_dict.items() if any(kw.lower() in combined_abstract for kw in kw_list)]  # Match categories

    # If still no categories matched, return "Uncategorized"
    return matched_categories if matched_categories else ["Uncategorized"]

In [20]:
# Main Function
def main():
    input_file = r"/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data/ICML.xlsx"
    df = pd.read_excel(input_file)  # Read Excel file

    # Fill missing values
    df["title"] = df["title"].fillna("")  
    df["keywords"] = df["keywords"].fillna("")  
    df["abstract"] = df["abstract"].fillna("")  

    # Apply original classification
    df["Predicted_Categories"] = df.apply(lambda row: ", ".join(classify_text(
        row["title"], row["keywords"], row["abstract"], CATEGORIES_KEYWORDS)), axis=1)  # Apply original classification method

    # Apply new classification
    df["Predicted_Categories2"] = df.apply(lambda row: ", ".join(classify_text(
        row["title"], row["keywords"], row["abstract"], ADDITIONAL_CATEGORIES_KEYWORDS)), axis=1)  # Apply new classification method

    # Save results to a new Excel file
    output_file = r"/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/ICML_categories.xlsx"
    df.to_excel(output_file, index=False)  # Save results to Excel file

    print("Classification completed, results have been saved to:", output_file)  # Print the output path
# Running the Main Function
if __name__ == "__main__":
    main()  # Run the main function


Classification completed, results have been saved to: /Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/ICML_categories.xlsx


In [21]:
# Read data
file_path = "/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/ICML_categories.xlsx"
df = pd.read_excel(file_path)

In [22]:
# Define Target Categories
category_columns = [
    "General Machine Learning", "Deep Learning", "Evaluation", "Theory of Machine Learning",
    "Machine Learning Systems", "Optimization", "Probabilistic Methods", "Reinforcement Learning",
    "Trustworthy Machine Learning", "Application-Driven Machine Learning"
]

In [23]:
# Count Articles per Category for Each Year
category_counts = df.groupby("year")["Predicted_Categories"].apply(lambda x: x.str.split(", ").explode().value_counts()).unstack(fill_value=0)

In [24]:
# Calculate Total Articles per Year
yearly_counts = df["year"].value_counts().sort_index()

In [25]:
# Calculate Relative Frequency of Each Category per Year
category_freq = category_counts.div(yearly_counts, axis=0)

In [26]:
# Plot the Trends
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")
sns.set_palette("tab10")

for category in category_columns:
    if category in category_freq.columns:
        sns.lineplot(x=category_freq.index, y=category_freq[category], label=category, linewidth=2)
# Set X-axis Range and Labels
plt.xlim(2020, 2024)
plt.xticks([2020, 2021, 2022, 2023, 2024], fontsize=12)  # Ensure x-axis only shows 2020-2024
plt.yticks(fontsize=12)
# Add Title and Labels
plt.title("Trends of Machine Learning Topics in ICML (2020-2024)", fontsize=16, fontweight='bold')
plt.xlabel("Year", fontsize=14)
plt.ylabel("Relative Frequency", fontsize=14)
# Adjust Legend Position
plt.legend(title="ML Categories", bbox_to_anchor=(1.02, 1), loc="upper left", fontsize=12)
# Add Grid Lines
plt.grid(True, linestyle="--", alpha=0.7)
# Automatically Adjust Layout
plt.tight_layout()
# Save the Plot as a PNG File
save_path = "/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Code files_Junbo/Result/Picture/Trends in ML Research Topics Over Time_python.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')  # Save as PNG file
plt.show()

In [11]:
import pandas as pd
import string
from collections import defaultdict
import spacy  # 改用spacy提升处理效率
from gensim.models import Phrases, LdaModel, TfidfModel
from gensim.corpora import Dictionary
from gensim.models.phrases import Phraser

# 配置参数
CONFIG = {
    "input_file": "/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data/ICML.xlsx",
    "output_file": "/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data_labels/ICML_LDA_categories.xlsx",
    "min_bigram_count": 5,
    "bigram_threshold": 10,
    "num_topics": 15,  # 增加主题数以提高区分度
    "lda_passes": 15,
    "top_keywords": 8,  # 每篇文章提取的关键词数量
    "category_threshold": 0.2  # 多标签分类阈值
}

# 预处理后的分类关键词（使用下划线形式）
CATEGORIES_KEYWORDS = {
    "General Machine Learning": [
        "general_machine_learning", "active_learning", "query_strategy", 
        "uncertainty_sampling", "clustering", "k_means", ...  # 保持原有结构，关键词改为下划线形式
    ],
    # 其他类别同理...
}

# 加载spacy模型（需提前运行 python -m spacy download en_core_web_sm）
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def preprocess_text(text):
    """增强版文本预处理"""
    if pd.isna(text) or not text.strip():
        return []
    
    # spaCy处理管道
    doc = nlp(text.lower().translate(str.maketrans('', '', string.punctuation)))
    return [
        token.lemma_ 
        for token in doc 
        if not token.is_stop 
        and token.is_alpha
        and len(token) > 2
    ]

def build_bigram_model(docs):
    """构建bigram模型"""
    bigram = Phrases(
        docs,
        min_count=CONFIG["min_bigram_count"],
        threshold=CONFIG["bigram_threshold"],
        connector_words=spacy.lang.en.STOP_WORDS
    )
    return Phraser(bigram)

def enhance_with_bigrams(docs, bigram_model):
    """将bigram合并到文本中"""
    return [bigram_model[doc] for doc in docs]

def train_lda_model(corpus_tfidf, dictionary):
    """训练LDA模型并自动选择最佳主题数"""
    return LdaModel(
        corpus_tfidf,
        id2word=dictionary,
        num_topics=CONFIG["num_topics"],
        passes=CONFIG["lda_passes"],
        alpha="auto",
        eta="auto",
        random_state=42
    )

def get_topic_keywords(bow, lda_model, dictionary, topn=5):
    """基于TF-IDF权重的关键词提取"""
    tfidf = TfidfModel(dictionary=dictionary)
    weights = {id: weight for id, weight in tfidf[bow]}
    
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0)
    dominant_topic = max(topic_dist, key=lambda x: x[1])[0]
    
    # 合并原始频率和TF-IDF权重
    topic_words = lda_model.get_topic_terms(dominant_topic, topn=topn*2)
    weighted_words = []
    for word_id, prob in topic_words:
        word = dictionary[word_id]
        tfidf_weight = weights.get(word_id, 0)
        weighted_words.append((word, prob * 0.7 + tfidf_weight * 0.3))
    
    return [word for word, _ in sorted(weighted_words, key=lambda x: -x[1])][:topn]

def assign_categories(keywords, categories):
    """多标签分类"""
    category_scores = defaultdict(float)
    for kw in keywords:
        for cat, terms in categories.items():
            if kw in terms:
                # 关键词在类别中的位置加权
                weight = 1.0 - (terms.index(kw)/len(terms)) * 0.5
                category_scores[cat] += weight
    
    max_score = max(category_scores.values(), default=0)
    return [
        cat for cat, score in category_scores.items()
        if score >= max_score * CONFIG["category_threshold"]
    ] or ["Uncategorized"]

# 主流程
if __name__ == "__main__":
    # 数据加载与预处理
    df = pd.read_excel(CONFIG["input_file"])
    df["processed"] = df["title"].fillna("") + " " + df["keywords"].fillna("") + " " + df["abstract"].fillna("")
    df["tokens"] = df["processed"].apply(preprocess_text)
    
    # Bigram处理
    bigram_model = build_bigram_model(df["tokens"])
    df["tokens"] = enhance_with_bigrams(df["tokens"], bigram_model)
    
    # 构建词典与语料
    dictionary = Dictionary(df["tokens"])
    dictionary.filter_extremes(no_below=5, no_above=0.5)  # 过滤低频/高频词
    corpus = [dictionary.doc2bow(doc) for doc in df["tokens"]]
    
    # TF-IDF转换
    tfidf = TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    
    # 训练LDA模型
    lda_model = train_lda_model(corpus_tfidf, dictionary)
    
    # 提取关键词
    df["core_keywords"] = [
        get_topic_keywords(doc, lda_model, dictionary, CONFIG["top_keywords"])
        for doc in corpus
    ]
    
    # 分类
    df["categories"] = df["core_keywords"].apply(
        lambda x: assign_categories(x, CATEGORIES_KEYWORDS)
    )
    
    # 保存结果
    df[["title", "categories", "core_keywords"]].to_excel(CONFIG["output_file"], index=False)
    
    # 结果验证
    print("\n分类结果示例:")
    print(df.sample(5)[["title", "categories"]])
    print(f"\n类别分布:\n{df['categories'].explode().value_counts()}")

/Users/alan/anaconda3/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data/ICML.xlsx'

In [2]:
pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 12.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.2/774.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.4/634.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 43.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [9]:
# 使用spacy自带的下载命令（推荐）
python -m spacy download en_core_web_sm

# 如果失败，尝试直接通过pip安装
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.0/en_core_web_sm-3.7.0-py3-none-any.whl

SyntaxError: invalid syntax (1216434129.py, line 2)

In [10]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.0/en_core_web_sm-3.7.0-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 3.2 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 1.2.0
    Uninstalling blis-1.2.0:
      Successfully uninstalled blis-1.2.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.4
    Uninstalling thinc-8.3.4:
      Successfully uninstalled thinc-8.3.4
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.4
    Uninstalling spacy-3.8.4:
      Successfully uninstalled spacy-3.8.4
Note: you may need to restart the kernel to use updated packages.
